# 通义千问使用函数调用

理论上最好的实现方式是配置好api直接就能函数调用，但是有一些api他跟openai的函数调用格式不一样，因此给出一个通用的方式进行函数调用。

以通义千问为例，参考文档：https://help.aliyun.com/zh/model-studio/developer-reference/use-qwen-by-calling-api?spm=a2c4g.11186623.0.0.459619a1mT8CvR#b5e1375f75n6f


首先创建两个agent：

In [6]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

# 加载LLM配置
config_list = config_list_from_json(env_or_file="qianwen_config.json")# 和openai的config格式一样

# 指定docker镜像名称
use_docker = False

# 创建了一个名为“assistant”的AssistantAgent实例，作为助手
assistant = AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 10,  # seed for caching and reproducibility
        "config_list": config_list,
        "temperature": 0,  # temperature for sampling
    },
)

# 创建了名为“user”的UserProxyAgent实例，作为人类用户的代理
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS", 
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), # 消息以TERMINATE结尾时结束对话
    code_execution_config={
        "work_dir": "coding", # 生成的文件和代码都会保存在这个目录
        "use_docker":use_docker, # 设置镜像名称
        "timeout": 60, # 执行代码的最大秒数
        "last_n_messages": 1 # 执行代码参考前面多少条消息
    }
)


In [7]:
def ask_expert(message):
    
    # 创建专家助手
    assistant_for_expert = AssistantAgent(
        name="assistant_for_expert",
        system_message="你是一个有用的助理。当需要你编写代码时，请先给出配置环境的shell脚本，在用户反馈配置好环境后，再给出对应代码。",
        llm_config={
            "temperature": 0,
            "config_list": config_list,
        },
    )
    
    # 创建专家
    expert = UserProxyAgent(
        name="expert",
        human_input_mode="ALWAYS",
        code_execution_config={"work_dir": working_directory, "use_docker": use_docker},
    )

    # 专家将收到的问题交给助手
    expert.initiate_chat(assistant_for_expert, message=message)
    
    # 专家在接收到助手的消息之后不再自动回复
    expert.stop_reply_at_receive(assistant_for_expert)

    # 输入exit后，专家结束对话并发送这条消息来让助手总结解法
    expert.send("总结解法并用简单易懂的方式说明答案", assistant_for_expert)
    
    # 返回专家收到的最后一条消息
    return expert.last_message()["content"]

定义函数和调用：

In [8]:
from datetime import datetime
from openai import OpenAI
import os
import random
import json

client = OpenAI(
        api_key=config_list[0]["api_key"], # 千问的api key
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 定义工具列表，模型在选择使用哪个工具时会参考工具的name和description
tools = [
    # 工具1 获取当前时刻的时间
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {}  # 因为获取当前时间无需输入参数，因此parameters为空字典
        }
    },
    # 工具2 获取指定城市的天气
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {  # 查询天气时需要提供位置，因此参数设置为location
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。"
                    }
                }
            },
            "required": [
                "location"
            ]
        }
    },
    {
        "type": "function",
        "function":{
            "name": "ask_expert",
            "description": "当你无法解决好问题时将问题交给专家.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "询问专家的问题. 确保问题包含足够的内容, 例如代码和执行结果. 专家不知道你和用户之间的对话，除非你把对话分享给专家",
                    }
                }
            },
            "required": [
                "message"
            ]
        }
    }
]


# 模拟天气查询工具。返回结果示例：“北京今天是晴天。”
def get_current_weather(location):
    return f"{location}今天是晴天。 "


# 查询当前时间的工具。返回结果示例：“当前时间：2024-04-15 17:15:18。“
def get_current_time():
    # 获取当前日期和时间
    current_datetime = datetime.now()
    # 格式化当前日期和时间
    formatted_time = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    # 返回格式化后的当前时间
    return f"当前时间：{formatted_time}。"

def call_with_messages(recipient, messages, sender, config):
    first_response = client.chat.completions.create(
        model="qwen-plus",
        messages=messages,
        tools=tools
    )
    assistant_output = first_response.choices[0].message
    
    print(f"\noutput of First round: {first_response}\n")
    # If the model determines that there is no need to invoke the tool, then print out the assistant"s response directly without making a second call to the model.

    # 如果模型认为不需要调用函数，就直接返回
    if assistant_output.tool_calls is None:
        return True, assistant_output.content
    else:
        # 将返回的函数调用加入消息列表
        messages.append(assistant_output)
        # 提取调用的函数名
        function_name = assistant_output.tool_calls[0].function.name
        # 如果调用get_current_weather
        if function_name == "get_current_weather":
            tool_info = {"name": "get_current_weather", "role": "tool"}
            location = json.loads(assistant_output.tool_calls[0].function.arguments)["properties"]["location"]
            tool_info["content"] = get_current_weather(location) # 将模型回复放在tool_info["content"]中
        # 如果调用get_current_time
        elif function_name == "get_current_time":
            tool_info = {"name": "get_current_time", "role": "tool"}
            tool_info["content"] = get_current_time()
        elif function_name == "ask_expert":
            tool_info = {"name": "ask_expert", "role": "tool"}
            location = json.loads(assistant_output.tool_calls[0].function.arguments)["properties"]["message"]
            tool_info["content"] = ask_expert(message)
    # 输出函数回复
    print(f"Tool info：{tool_info['content']}\n")

    # 将函数输出作为message放入对话历史中
    messages.append({
        "role": "tool",
        "name": function_name,
        "content": tool_info["content"],
        "tool_call_id":assistant_output.tool_calls[0].id
    })

    # 模型总结函数的输出
    second_response = client.chat.completions.create(
        model="qwen-plus",
        messages=messages,
    )
    print(f"Output of second round: {second_response}\n")
    print(f"Final response: {second_response.choices[0].message.content}")
    
    return True, second_response.choices[0].message.content

In [9]:
# 注册回复函数
assistant.register_reply(
    trigger=[autogen.Agent, None], 
    reply_func=call_with_messages, 
    # position = 0, # 
    config={},
)

In [10]:
user_proxy.initiate_chat(
    assistant,
    message="""# 假设 x + y != -1
# 方程组
# ax + by + c = x + 7
# a + bx + cy = 2x + 6y
# ay + b + cx = 4x + y
# 求 a + b + c"""
)
# 请问现在是几点？
# 今天上海是什么天气？

user_proxy (to assistant):

# 假设 x + y != -1
# 方程组
# ax + by + c = x + 7
# a + bx + cy = 2x + 6y
# ay + b + cx = 4x + y
# 求 a + b + c

--------------------------------------------------------------------------------

output of First round: ChatCompletion(id='chatcmpl-af3618f6-ece1-9e66-858e-cdaebc3b9ef0', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='为了求解 \\(a + b + c\\), 我们首先需要对方程组进行简化和整理。给定的方程组如下：\n\n1. \\(ax + by + c = x + 7\\)\n2. \\(a + bx + cy = 2x + 6y\\)\n3. \\(ay + b + cx = 4x + y\\)\n\n我们可以通过整理这些方程来找出 \\(a\\)、\\(b\\) 和 \\(c\\) 的值。让我们先对每个方程重新组织，使其成为关于 \\(a\\)、\\(b\\) 和 \\(c\\) 的线性方程组形式。', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0071632b9b19407d9fe179', function=Function(arguments='{"properties": {"message": "我们需要求解一个线性方程组来找到 a, b 和 c 的值。\\n\\n方程组如下：\\n1. \\(ax + by + c = x + 7\\)\\n2. \\(a + bx + cy = 2x + 6y\\)\\n3. \\(ay + b + cx = 4x + y\\)\\n\\n我们的目标是求解 a, b 和 c 的值

JSONDecodeError: Invalid \escape: line 1 column 73 (char 72)